In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for i, word in enumerate(words):
        vocabs.append(word[0])
print(len(vocabs))
word_to_id = {word: i for i, word in enumerate(vocabs)}
id_to_word = {i:word for i, word in enumerate(vocabs)}

22673


In [3]:
id_to_word[0], word_to_id['<PAD>']

('<PAD>', 0)

In [4]:
hidden_size=100
sequence=5
embedding_dim=50
attention_size = 50
batch_size=8
vocab_size=len(vocabs)

In [5]:
def get_model_inputs():
    X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
    # Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
    Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
#     seq_len = tf.placeholder(tf.int32, [None], name='seq_len')
    lr = tf.placeholder(tf.float32, name='learning_rate')

#     embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
#     batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return X, Y, lr, target_sequence_length, source_sequence_length, max_target_sequence_length

In [6]:
def encoding_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

In [7]:
def process_decoder_input(target_data, word_to_int, batch_size):
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size,1], word_to_int['<GO>']), ending], 1)
    return dec_input

In [8]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, enc_state, dec_input):
    
    target_vocab_size = len(target_letter_to_int)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))

    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    output_layer = core.Dense(target_vocab_size,
                              kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
                             )
    
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length = target_sequence_length,
                                                            time_major=False
                                                           )
        
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           enc_state,
                                                           output_layer
                                                          )
        
        training_decoder_output,final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                                                        impute_finished=True,
                                                                                                        maximum_iterations=max_target_sequence_length
                                                                                                       )
        
    with tf.variable_scope('decode', reuse=True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']],dtype=tf.int32), [batch_size], name='start_tokens')
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                    start_tokens,
                                                                    target_letter_to_int['<EOS>']
                                                                   )
        
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                            inference_helper,
                                                            enc_state,
                                                            output_layer
                                                           )
        
        inference_decoder_output, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                                                          impute_finished=True,
                                                                                                          maximum_iterations=max_target_sequence_length
                                                                                                         )
        
        print('training_decoder_output.rnn_output, inference_decoder_output.sample_id', training_decoder_output.rnn_output, inference_decoder_output.sample_id)
        print('training_decoder_output, inference_decoder_output', training_decoder_output, inference_decoder_output)
    return training_decoder_output, inference_decoder_output

In [9]:
def seq2seq_model(input_data, targets, target_sequence_length,max_target_sequence_length,
                  source_sequence_length,source_vocab_size, target_vocab_size, enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers
                 ):

    _, enc_state = encoding_layer(input_data=input_data, rnn_size=rnn_size, num_layers=num_layers,
                                  source_sequence_length = source_sequence_length,
                                  source_vocab_size = source_vocab_size,
                                  encoding_embedding_size=embedding_dim
                                 )
    
    dec_input = process_decoder_input(targets, word_to_id, batch_size)

    training_decoder_output, inference_decoder_output = decoding_layer(target_letter_to_int=word_to_id,
                                                                       decoding_embedding_size=embedding_dim,
                                                                       num_layers = num_layers,
                                                                       rnn_size=rnn_size,
                                                                       target_sequence_length=target_sequence_length,
                                                                       max_target_sequence_length=max_target_sequence_length,
                                                                       enc_state=enc_state,
                                                                       dec_input=dec_input
                                                                      )
    
    return training_decoder_output,inference_decoder_output

In [10]:
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():

    # Load the model inputs    
    input_data, targets, lr, target_sequence_length, source_sequence_length, max_target_sequence_length = get_model_inputs()
    print('input_data fetched ')
    # Create the training and inference logits
    training_decoder_output, inference_decoder_output = seq2seq_model(input_data=input_data, 
                                                                      targets=targets, 
                                                                      target_sequence_length = target_sequence_length, 
                                                                      max_target_sequence_length = max_target_sequence_length, 
                                                                      source_sequence_length = source_sequence_length,
                                                                      source_vocab_size = len(word_to_id),
                                                                      target_vocab_size = len(word_to_id),
                                                                      enc_embedding_size = embedding_dim, 
                                                                      dec_embedding_size = embedding_dim, 
                                                                      rnn_size = hidden_size, 
                                                                      num_layers = 1)
    
    print('seq2seq model created! ')
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predictions_sample_id = tf.identity(inference_decoder_output.sample_id, name='predictions_sample_id')
    predictions_output = tf.identity(inference_decoder_output.rnn_output, name='predictions_output')

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):

        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
        print('optmization created!')

input_data fetched 
training_decoder_output.rnn_output, inference_decoder_output.sample_id Tensor("decode/decoder/transpose:0", shape=(8, ?, 22673), dtype=float32) Tensor("decode_1/decoder/transpose_1:0", shape=(8, ?), dtype=int32)
training_decoder_output, inference_decoder_output BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/transpose:0' shape=(8, ?, 22673) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/transpose_1:0' shape=(8, ?) dtype=int32>) BasicDecoderOutput(rnn_output=<tf.Tensor 'decode_1/decoder/transpose:0' shape=(8, ?, 22673) dtype=float32>, sample_id=<tf.Tensor 'decode_1/decoder/transpose_1:0' shape=(8, ?) dtype=int32>)
seq2seq model created! 
optmization created!


In [11]:
# Ses = tf.Session(graph=train_graph)
with tf.Session(graph=train_graph) as Ses:
    print('session !')
    file_name_queue = tf.train.string_input_producer(['inputs'])
    reader = tf.TFRecordReader()
    _, single_x = reader.read(file_name_queue)
    print('gogo~')
    context_features = {
        "length": tf.FixedLenFeature([], dtype=tf.int64)
    }

    sequence_features = {
        "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
        "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }

    context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                                                       context_features=context_features,
                                                                       sequence_features=sequence_features
                                                                      )
    context_parsed['length'] = tf.cast(context_parsed['length'], tf.int32)
    batch_seq, batch_inputs = tf.contrib.training.bucket_by_sequence_length(context_parsed['length'],
                                                                            [sequence_parsed['tokens'], sequence_parsed['labels']],
                                                                            batch_size=batch_size,
                                                                            bucket_boundaries=[5,10],
                                                                            dynamic_pad=True,
                                                                            capacity=1000,
                                                                            num_threads=1
                                                                           )

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
    Ses.run(tf.global_variables_initializer())
    epochs = 3
    for epoch in range(1, epochs):
        print('epoch', epoch)
        i = 0
        while True:

            seq_input_, batch_input_ = Ses.run([batch_seq, batch_inputs])

            feed_data = {input_data: batch_input_[0], targets: batch_input_[1], 
                         target_sequence_length:seq_input_, source_sequence_length:seq_input_, lr:0.0001}
            _, loss = Ses.run([train_op, cost], feed_data)
    #             print('started')
            
            i+=1
            print('{} loss {} '.format(i, loss))
            if i % 500 ==0:
                checkpoint = './seq2seq_test/seq2seq_test.ckpt'
                saver = tf.train.Saver()
                saver.save(Ses, checkpoint)
                print('model saved')
#                 break

session !
gogo~
epoch 1
1 loss 10.034784317016602 
2 loss 10.049150466918945 
3 loss 10.038866996765137 
4 loss 10.035192489624023 
5 loss 10.035387992858887 
6 loss 10.018266677856445 
7 loss 10.003318786621094 
8 loss 10.0033597946167 
9 loss 10.023435592651367 
10 loss 9.996979713439941 
11 loss 10.007317543029785 
12 loss 9.997413635253906 
13 loss 9.998591423034668 
14 loss 10.028757095336914 
15 loss 10.007110595703125 
16 loss 9.989767074584961 
17 loss 9.992267608642578 
18 loss 9.997990608215332 
19 loss 9.97113037109375 
20 loss 9.975897789001465 
21 loss 9.983406066894531 
22 loss 9.977084159851074 
23 loss 9.947110176086426 
24 loss 9.967721939086914 
25 loss 9.97146224975586 
26 loss 9.958761215209961 
27 loss 9.952962875366211 
28 loss 9.954035758972168 
29 loss 9.941047668457031 
30 loss 9.953229904174805 
31 loss 9.960759162902832 
32 loss 9.945850372314453 
33 loss 9.92403793334961 
34 loss 9.946723937988281 
35 loss 9.935462951660156 
36 loss 9.944519996643066 
37 los

298 loss 6.959563255310059 
299 loss 5.807003021240234 
300 loss 5.551800727844238 
301 loss 5.238955497741699 
302 loss 5.175642967224121 
303 loss 5.002407073974609 
304 loss 6.292495250701904 
305 loss 5.708924293518066 
306 loss 6.017657279968262 
307 loss 5.583518028259277 
308 loss 5.454006671905518 
309 loss 5.70459508895874 
310 loss 5.627336025238037 
311 loss 4.939162254333496 
312 loss 5.662390232086182 
313 loss 5.150076389312744 
314 loss 5.477194309234619 
315 loss 4.925417900085449 
316 loss 5.413773059844971 
317 loss 5.168546199798584 
318 loss 4.95095682144165 
319 loss 5.37235164642334 
320 loss 5.396090507507324 
321 loss 5.1847710609436035 
322 loss 5.431885719299316 
323 loss 5.24728536605835 
324 loss 4.967365741729736 
325 loss 4.944811820983887 
326 loss 5.234925746917725 
327 loss 5.218057155609131 
328 loss 4.704196453094482 
329 loss 5.292611122131348 
330 loss 4.876392364501953 
331 loss 4.799794673919678 
332 loss 5.282724857330322 
333 loss 5.087628841400

591 loss 4.847474575042725 
592 loss 4.265265464782715 
593 loss 5.632613182067871 
594 loss 5.614671230316162 
595 loss 5.771812915802002 
596 loss 6.069332122802734 
597 loss 5.642695426940918 
598 loss 4.491910457611084 
599 loss 4.91537618637085 
600 loss 4.445346355438232 
601 loss 4.697028636932373 
602 loss 4.29803991317749 
603 loss 4.002286911010742 
604 loss 4.85023832321167 
605 loss 5.063210487365723 
606 loss 4.148196220397949 
607 loss 5.321043968200684 
608 loss 5.106379508972168 
609 loss 5.220791339874268 
610 loss 5.50684118270874 
611 loss 5.483604431152344 
612 loss 5.250154972076416 
613 loss 4.357458114624023 
614 loss 4.786959648132324 
615 loss 4.588719367980957 
616 loss 5.137162208557129 
617 loss 6.024609565734863 
618 loss 5.989595413208008 
619 loss 5.413596153259277 
620 loss 4.864884376525879 
621 loss 4.791825294494629 
622 loss 4.822630882263184 
623 loss 4.540453910827637 
624 loss 5.124920845031738 
625 loss 5.643796920776367 
626 loss 4.9915461540222

885 loss 5.177959442138672 
886 loss 4.971560478210449 
887 loss 5.587465763092041 
888 loss 5.391226768493652 
889 loss 4.8289055824279785 
890 loss 5.203433513641357 
891 loss 5.19844913482666 
892 loss 5.840429306030273 
893 loss 4.524636268615723 
894 loss 4.840573310852051 
895 loss 4.5964674949646 
896 loss 5.379979133605957 
897 loss 5.1860246658325195 
898 loss 5.204575061798096 
899 loss 5.613672733306885 
900 loss 4.983118534088135 
901 loss 5.345621585845947 
902 loss 5.343236923217773 
903 loss 5.15049934387207 
904 loss 5.014115810394287 
905 loss 5.349403381347656 
906 loss 5.225693225860596 
907 loss 4.8230462074279785 
908 loss 5.05355978012085 
909 loss 5.039068698883057 
910 loss 4.337990760803223 
911 loss 5.131594657897949 
912 loss 4.5388593673706055 
913 loss 4.913023948669434 
914 loss 5.461496829986572 
915 loss 5.995785236358643 
916 loss 5.557724475860596 
917 loss 5.414236545562744 
918 loss 4.895645618438721 
919 loss 6.0856733322143555 
920 loss 5.332944393

1171 loss 4.071303367614746 
1172 loss 4.9474687576293945 
1173 loss 4.2958221435546875 
1174 loss 4.349475383758545 
1175 loss 4.099308967590332 
1176 loss 4.6464056968688965 
1177 loss 4.296070098876953 
1178 loss 4.263489723205566 
1179 loss 4.888036251068115 
1180 loss 4.135377407073975 
1181 loss 3.9415602684020996 
1182 loss 4.082257270812988 
1183 loss 4.34262228012085 
1184 loss 5.009732723236084 
1185 loss 4.499800205230713 
1186 loss 4.8255791664123535 
1187 loss 4.782474994659424 
1188 loss 4.521753787994385 
1189 loss 4.86665678024292 
1190 loss 4.687705039978027 
1191 loss 5.1507463455200195 
1192 loss 4.6859917640686035 
1193 loss 4.9029130935668945 
1194 loss 4.600692272186279 
1195 loss 4.7742180824279785 
1196 loss 4.363494873046875 
1197 loss 3.808872938156128 
1198 loss 4.417508602142334 
1199 loss 4.6918463706970215 
1200 loss 4.910073757171631 
1201 loss 4.657532691955566 
1202 loss 4.582420825958252 
1203 loss 4.796953201293945 
1204 loss 4.865420341491699 
1205 l

1454 loss 4.748547554016113 
1455 loss 4.860159873962402 
1456 loss 5.855720520019531 
1457 loss 5.24618673324585 
1458 loss 5.079704761505127 
1459 loss 5.167721271514893 
1460 loss 4.881958961486816 
1461 loss 4.59659481048584 
1462 loss 5.343807220458984 
1463 loss 4.273881912231445 
1464 loss 4.562141418457031 
1465 loss 4.901833534240723 
1466 loss 5.739407062530518 
1467 loss 5.014712333679199 
1468 loss 4.940277576446533 
1469 loss 5.336703777313232 
1470 loss 5.487195014953613 
1471 loss 5.262092113494873 
1472 loss 5.3685102462768555 
1473 loss 4.8727617263793945 
1474 loss 4.071681976318359 
1475 loss 4.712046146392822 
1476 loss 4.784842491149902 
1477 loss 3.8326478004455566 
1478 loss 4.359503269195557 
1479 loss 4.668839931488037 
1480 loss 5.1208648681640625 
1481 loss 5.147295951843262 
1482 loss 5.130565643310547 
1483 loss 5.0879316329956055 
1484 loss 5.122311592102051 
1485 loss 4.372041702270508 
1486 loss 4.356964111328125 
1487 loss 4.25115966796875 
1488 loss 4.

1736 loss 4.936851978302002 
1737 loss 5.3648457527160645 
1738 loss 4.426651954650879 
1739 loss 4.253162384033203 
1740 loss 5.227593421936035 
1741 loss 4.45328426361084 
1742 loss 5.384765148162842 
1743 loss 4.852837562561035 
1744 loss 5.041385650634766 
1745 loss 4.395454406738281 
1746 loss 5.233083724975586 
1747 loss 5.3826680183410645 
1748 loss 4.922812461853027 
1749 loss 5.031113147735596 
1750 loss 5.851683139801025 
1751 loss 5.7215142250061035 
1752 loss 6.565346717834473 
1753 loss 4.323275089263916 
1754 loss 5.692357063293457 
1755 loss 4.76231050491333 
1756 loss 4.566586494445801 
1757 loss 3.8672218322753906 
1758 loss 4.48446798324585 
1759 loss 5.269690990447998 
1760 loss 5.332113742828369 
1761 loss 5.016870975494385 
1762 loss 4.90918493270874 
1763 loss 5.348015785217285 
1764 loss 4.975821018218994 
1765 loss 4.827958106994629 
1766 loss 6.015442848205566 
1767 loss 6.120729923248291 
1768 loss 4.5588483810424805 
1769 loss 5.033917427062988 
1770 loss 4.6

2020 loss 4.773831367492676 
2021 loss 4.929027080535889 
2022 loss 5.2366743087768555 
2023 loss 5.453485012054443 
2024 loss 5.40997838973999 
2025 loss 4.734894752502441 
2026 loss 5.197879791259766 
2027 loss 5.156147480010986 
2028 loss 3.5891261100769043 
2029 loss 3.205656051635742 
2030 loss 3.1817705631256104 
2031 loss 4.1162590980529785 
2032 loss 2.9000778198242188 
2033 loss 2.871094226837158 
2034 loss 2.8379299640655518 
2035 loss 2.8013663291931152 
2036 loss 2.762120485305786 
2037 loss 2.720825672149658 
2038 loss 2.6780612468719482 
2039 loss 2.6343235969543457 
2040 loss 2.5900497436523438 
2041 loss 2.545605182647705 
2042 loss 2.501298427581787 
2043 loss 2.45737886428833 
2044 loss 4.154560089111328 
2045 loss 5.901302337646484 
2046 loss 5.419339656829834 
2047 loss 4.595764636993408 
2048 loss 5.583353042602539 
2049 loss 6.2510199546813965 
2050 loss 5.615978717803955 
2051 loss 4.8200554847717285 
2052 loss 5.786318302154541 
2053 loss 5.740091800689697 
2054

2302 loss 4.822567939758301 
2303 loss 4.895260810852051 
2304 loss 4.600921630859375 
2305 loss 4.502542972564697 
2306 loss 4.268848419189453 
2307 loss 5.339853763580322 
2308 loss 4.778702735900879 
2309 loss 5.029746055603027 
2310 loss 5.145140647888184 
2311 loss 4.862133979797363 
2312 loss 5.437142848968506 
2313 loss 5.95711612701416 
2314 loss 4.133976936340332 
2315 loss 4.9518609046936035 
2316 loss 4.10414981842041 
2317 loss 4.422328948974609 
2318 loss 5.0725507736206055 
2319 loss 5.936371803283691 
2320 loss 5.77449893951416 
2321 loss 5.5583815574646 
2322 loss 5.9215521812438965 
2323 loss 7.133267402648926 
2324 loss 6.373517990112305 
2325 loss 7.065405368804932 
2326 loss 7.197000980377197 
2327 loss 6.286139488220215 
2328 loss 7.1038641929626465 
2329 loss 6.641026973724365 
2330 loss 6.5894646644592285 
2331 loss 7.219405174255371 
2332 loss 5.815951347351074 
2333 loss 7.0936079025268555 
2334 loss 6.809689521789551 
2335 loss 6.57512903213501 
2336 loss 6.75

KeyboardInterrupt: 